# Introduction

This notebook provides a walkthrough of some of the functionality available for users looking to create and maintain large databases using PyKX.

In particular, this notebook refers to creating and maintaining [partitioned kdb+ databases](https://code.kx.com/q/kb/partition/). Go to [Q for Mortals](https://code.kx.com/q4m3/14_Introduction_to_Kdb+/#143-partitioned-tables) for more in-depth information about partitioned databases in kdb+.

You can download this walkthrough as a `.ipynb` notebook file using the following <a href="./db-management.ipynb" download>link</a>.
This walkthrough provides examples of the following tasks:

1. Creating a database from a historical dataset
1. Adding a new partition to the database
1. Managing the on-disk database by:
  1. Renaming a table and column
  2. Creating a copy of a column to the database
  3. Applying a Python function to a column of the database
  4. Updating the data type of a column
1. Adding a new table to the most recent partition of the database

For full information on the functions available you can reference the [API section](https://code.kx.com/pykx/api/db.html).

---

## Initial setup

Import all required libraries and create a temporary directory which will be used to store the database we create for this walkthrough

In [1]:
import os
os.environ['PYKX_BETA_FEATURES'] = 'true'

import pykx as kx
from datetime import date
import tempfile

In [2]:
tempdir = tempfile.TemporaryDirectory()

Database interactions are facilitated through use of the `pykx.DB` class. All methods/attributes used in this notebook are contained within this class. 

Initialise the `DB` class to start. The expected input is the file path where you intend to save the partitioned database and its associated tables. In this case we're going to use the temporary directory we just created. 

In [3]:
db = kx.DB(path = tempdir.name + '/db')

For details on any methods contained within this class, you can use the `help` method. 

In [4]:
help(db.create)

Help on method create in module pykx.db:

create(table, table_name, partition, *, by_field=None, sym_enum=None, log=True) method of pykx.db.DB instance
    Create an on-disk partitioned table within a kdb+ database from a supplied
        `pykx.Table` object. Once generated this table will be accessible
        as an attribute of the `DB` class or a sub attribute of `DB.table`.
    
    Parameters:
        table: The `pykx.Table` object which is to be persisted to disk
        table_name: The name with which the table will be persisted and accessible
            once loaded and available as a `pykx.PartitionedTable`
        partition: The name of the column which is to be used to partition the data if
            supplied as a `str` or if supplied as non string object this will be used as
            the partition to which all data is persisted
        by_field: A field of the table to be used as a by column, this column will be
            the second column in the table (the first bei

---

## Create the sample dataset

Create a dataset called `trades` containing time-series data spanning multiple dates, and columns of various types:

In [5]:
N = 1000000
trades = kx.Table(data={
     'date': kx.random.random(N, [date(2020, 1, 1), date(2020, 1, 2)]),
     'sym': kx.random.random(N, ['AAPL', 'GOOG', 'MSFT']),
     'price': kx.random.random(N, 10.0),
     'size': kx.random.random(N, 1000)
})

---

## Create the database

Create the database using the `date` column as the partition, and add `trades` as a table called `trade_data` within it.

In [6]:
db.create(trades, 'trade_data', 'date')

Writing Database Partition 2020.01.01 to table trade_data
Writing Database Partition 2020.01.02 to table trade_data


This now exists as a table and is saved to disk.

In [7]:
db.tables

['trade_data']

When a table is saved, an attribute is added to the `db` class for it. For our newly generated table, this is `db.trade_data`

In [8]:
db.trade_data

,date,sym,price,size
,,,,
0,2020.01.01,MSFT,7.079266,800
1,2020.01.01,AAPL,1.824321,65
2,2020.01.01,MSFT,2.408259,292
3,2020.01.01,GOOG,1.675438,7
4,2020.01.01,AAPL,8.311168,183
5,2020.01.01,AAPL,2.208693,989
6,2020.01.01,MSFT,6.068126,567
7,2020.01.01,AAPL,4.918926,794
8,2020.01.01,AAPL,9.331869,39


---

## Add a new partition to the database

Once a table has been generated, you can add more partitions to the database through reuse of the `create` method. In this case we are adding the new partition `2020.01.03` to the database.

In [9]:
N = 10000
new_day = kx.Table(data={
    'sym': kx.random.random(N, ['AAPL', 'GOOG', 'MSFT']),
    'price': kx.random.random(N, 10.0),
    'size': kx.random.random(N, 100)
})
db.create(new_day, 'trade_data', date(2020, 1, 3))

Writing Database Partition 2020-01-03 to table trade_data


---

## Manage the database

This section covers updating the contents of a database. The examples below demonstrate a number of common tasks that would be completed regularly when updating a database.

The name of a table can be updated using the `rename_table` method. Below, we are updating the table `trade_data` to be called `trade`.

In [10]:
db.rename_table('trade_data', 'trades')

2023.12.15 16:14:22 renaming :/var/folders/l8/t7s11kcs02x3dchm9_m48mq80000gn/T/tmp2ts68edc/db/2020.01.01/trade_data to :/var/folders/l8/t7s11kcs02x3dchm9_m48mq80000gn/T/tmp2ts68edc/db/2020.01.01/trades
2023.12.15 16:14:22 renaming :/var/folders/l8/t7s11kcs02x3dchm9_m48mq80000gn/T/tmp2ts68edc/db/2020.01.02/trade_data to :/var/folders/l8/t7s11kcs02x3dchm9_m48mq80000gn/T/tmp2ts68edc/db/2020.01.02/trades
2023.12.15 16:14:22 renaming :/var/folders/l8/t7s11kcs02x3dchm9_m48mq80000gn/T/tmp2ts68edc/db/2020.01.03/trade_data to :/var/folders/l8/t7s11kcs02x3dchm9_m48mq80000gn/T/tmp2ts68edc/db/2020.01.03/trades


During the rename process, the attribute in the `db` class is also updated. 

In [11]:
db.trades

,date,sym,price,size
,,,,
0,2020.01.01,MSFT,7.079266,800
1,2020.01.01,AAPL,1.824321,65
2,2020.01.01,MSFT,2.408259,292
3,2020.01.01,GOOG,1.675438,7
4,2020.01.01,AAPL,8.311168,183
5,2020.01.01,AAPL,2.208693,989
6,2020.01.01,MSFT,6.068126,567
7,2020.01.01,AAPL,4.918926,794
8,2020.01.01,AAPL,9.331869,39


Renaming a column in a table is achieved using the `rename_column` method. For example, let's update the `sym` column in the `trade` table to be called `ticker`.

In [12]:
db.rename_column('trades', 'sym', 'ticker')

2023.12.15 16:14:25 renaming sym to ticker in `:/var/folders/l8/t7s11kcs02x3dchm9_m48mq80000gn/T/tmp2ts68edc/db/2020.01.01/trades
2023.12.15 16:14:25 renaming sym to ticker in `:/var/folders/l8/t7s11kcs02x3dchm9_m48mq80000gn/T/tmp2ts68edc/db/2020.01.02/trades
2023.12.15 16:14:25 renaming sym to ticker in `:/var/folders/l8/t7s11kcs02x3dchm9_m48mq80000gn/T/tmp2ts68edc/db/2020.01.03/trades


In [13]:
db.trades

,date,ticker,price,size
,,,,
0,2020.01.01,MSFT,7.079266,800
1,2020.01.01,AAPL,1.824321,65
2,2020.01.01,MSFT,2.408259,292
3,2020.01.01,GOOG,1.675438,7
4,2020.01.01,AAPL,8.311168,183
5,2020.01.01,AAPL,2.208693,989
6,2020.01.01,MSFT,6.068126,567
7,2020.01.01,AAPL,4.918926,794
8,2020.01.01,AAPL,9.331869,39


To safely apply a function to modify the `price` column within the database, first create a copy of the column.

In [14]:
db.copy_column('trades', 'price', 'price_copy')

2023.12.15 16:14:29 copying price to price_copy in `:/var/folders/l8/t7s11kcs02x3dchm9_m48mq80000gn/T/tmp2ts68edc/db/2020.01.01/trades
2023.12.15 16:14:29 copying price to price_copy in `:/var/folders/l8/t7s11kcs02x3dchm9_m48mq80000gn/T/tmp2ts68edc/db/2020.01.02/trades
2023.12.15 16:14:29 copying price to price_copy in `:/var/folders/l8/t7s11kcs02x3dchm9_m48mq80000gn/T/tmp2ts68edc/db/2020.01.03/trades


In [15]:
db.trades

,date,ticker,price,size,price_copy
,,,,,
0,2020.01.01,MSFT,7.079266,800,7.079266
1,2020.01.01,AAPL,1.824321,65,1.824321
2,2020.01.01,MSFT,2.408259,292,2.408259
3,2020.01.01,GOOG,1.675438,7,1.675438
4,2020.01.01,AAPL,8.311168,183,8.311168
5,2020.01.01,AAPL,2.208693,989,2.208693
6,2020.01.01,MSFT,6.068126,567,6.068126
7,2020.01.01,AAPL,4.918926,794,4.918926
8,2020.01.01,AAPL,9.331869,39,9.331869


You can now apply a function to the copied column without the risk of losing the original data. Below we are modifying the copied column by multiplying the contents by 2.

In [16]:
db.apply_function('trades', 'price_copy', kx.q('{2*x}'))

2023.12.15 16:14:31 resaving column price_copy (type 9) in `:/var/folders/l8/t7s11kcs02x3dchm9_m48mq80000gn/T/tmp2ts68edc/db/2020.01.01/trades
2023.12.15 16:14:31 resaving column price_copy (type 9) in `:/var/folders/l8/t7s11kcs02x3dchm9_m48mq80000gn/T/tmp2ts68edc/db/2020.01.02/trades
2023.12.15 16:14:31 resaving column price_copy (type 9) in `:/var/folders/l8/t7s11kcs02x3dchm9_m48mq80000gn/T/tmp2ts68edc/db/2020.01.03/trades


In [17]:
db.trades

,date,ticker,price,size,price_copy
,,,,,
0,2020.01.01,MSFT,7.079266,800,14.15853
1,2020.01.01,AAPL,1.824321,65,3.648642
2,2020.01.01,MSFT,2.408259,292,4.816519
3,2020.01.01,GOOG,1.675438,7,3.350875
4,2020.01.01,AAPL,8.311168,183,16.62234
5,2020.01.01,AAPL,2.208693,989,4.417385
6,2020.01.01,MSFT,6.068126,567,12.13625
7,2020.01.01,AAPL,4.918926,794,9.837851
8,2020.01.01,AAPL,9.331869,39,18.66374


Once you are happy with the new values within the `price_copy` column, you can safely delete the `price` column, then rename the `price_copy` column to be called `price`.

In [18]:
db.delete_column('trades', 'price')
db.rename_column('trades', 'price_copy', 'price')

2023.12.15 16:14:33 deleting column price from `:/var/folders/l8/t7s11kcs02x3dchm9_m48mq80000gn/T/tmp2ts68edc/db/2020.01.01/trades
2023.12.15 16:14:33 deleting column price from `:/var/folders/l8/t7s11kcs02x3dchm9_m48mq80000gn/T/tmp2ts68edc/db/2020.01.02/trades
2023.12.15 16:14:33 deleting column price from `:/var/folders/l8/t7s11kcs02x3dchm9_m48mq80000gn/T/tmp2ts68edc/db/2020.01.03/trades
2023.12.15 16:14:33 renaming price_copy to price in `:/var/folders/l8/t7s11kcs02x3dchm9_m48mq80000gn/T/tmp2ts68edc/db/2020.01.01/trades
2023.12.15 16:14:33 renaming price_copy to price in `:/var/folders/l8/t7s11kcs02x3dchm9_m48mq80000gn/T/tmp2ts68edc/db/2020.01.02/trades
2023.12.15 16:14:33 renaming price_copy to price in `:/var/folders/l8/t7s11kcs02x3dchm9_m48mq80000gn/T/tmp2ts68edc/db/2020.01.03/trades


In [19]:
db.trades

,date,ticker,size,price
,,,,
0,2020.01.01,MSFT,800,14.15853
1,2020.01.01,AAPL,65,3.648642
2,2020.01.01,MSFT,292,4.816519
3,2020.01.01,GOOG,7,3.350875
4,2020.01.01,AAPL,183,16.62234
5,2020.01.01,AAPL,989,4.417385
6,2020.01.01,MSFT,567,12.13625
7,2020.01.01,AAPL,794,9.837851
8,2020.01.01,AAPL,39,18.66374


To convert the data type of a column, you can use the `set_column_type` method. Before we do that, we can look at the metadata information for the table using the `meta` method. 


In [20]:
kx.q.meta(db.trades)

,t,f,a
c,,,
date,"""d""",,
ticker,"""s""",,
size,"""j""",,
price,"""f""",,


Currently the `size` column is the type `LongAtom`. We will update this to be a type `ShortAtom`.

In [21]:
db.set_column_type('trades', 'size', kx.ShortAtom)

2023.12.15 16:20:03 resaving column size (type 5) in `:/var/folders/l8/t7s11kcs02x3dchm9_m48mq80000gn/T/tmp2ts68edc/db/2020.01.01/trades
2023.12.15 16:20:03 resaving column size (type 5) in `:/var/folders/l8/t7s11kcs02x3dchm9_m48mq80000gn/T/tmp2ts68edc/db/2020.01.02/trades
2023.12.15 16:20:03 resaving column size (type 5) in `:/var/folders/l8/t7s11kcs02x3dchm9_m48mq80000gn/T/tmp2ts68edc/db/2020.01.03/trades


Now let's apply the `grouped` attribute to the size column. For more information on attributes in kdb+, please refer to the Q for Mortals [Attributes section](https://code.kx.com/q4m3/8_Tables/#88-attributes).

In [22]:
db.set_column_attribute('trades', 'ticker', 'grouped')

2023.12.15 16:20:04 resaving column ticker (type 20) in `:/var/folders/l8/t7s11kcs02x3dchm9_m48mq80000gn/T/tmp2ts68edc/db/2020.01.01/trades
2023.12.15 16:20:04 resaving column ticker (type 20) in `:/var/folders/l8/t7s11kcs02x3dchm9_m48mq80000gn/T/tmp2ts68edc/db/2020.01.02/trades
2023.12.15 16:20:04 resaving column ticker (type 20) in `:/var/folders/l8/t7s11kcs02x3dchm9_m48mq80000gn/T/tmp2ts68edc/db/2020.01.03/trades


Let's revisit the metadata of the table to ensure they have been applied correctly.

In [23]:
kx.q.meta(db.trades)

,t,f,a
c,,,
date,"""d""",,
ticker,"""s""",,g
size,"""h""",,
price,"""f""",,


## Onboarding your next table

Now that you have successfully set up one table, you may want to add a second table. We follow the same method as before and create the `quotes` table using the `create` method. In this example, the `quotes` table only contains data for `2020.01.03`.

In [24]:
quotes = kx.Table(data={
    'sym': kx.random.random(N, ['AAPL', 'GOOG', 'MSFT']),
    'open': kx.random.random(N, 10.0),
    'high': kx.random.random(N, 10.0),
    'low': kx.random.random(N, 10.0),
    'close': kx.random.random(N, 10.0)
})

In [25]:
db.create(quotes, 'quotes', date(2020, 1, 3), by_field = 'sym')

Writing Database Partition 2020-01-03 to table quotes


All tables within a database must contain the same partition structure. To ensure the new table can be accessed, the `quotes` table needs to exist in every partition within the database, even if there is no data for that partition. This is called backfilling data. For the partitions where the `quotes` table is missing, we use the `fill_database` method. 

In [26]:
db.fill_database()

Successfully filled missing tables to partition: :/var/folders/l8/t7s11kcs02x3dchm9_m48mq80000gn/T/tmp2ts68edc/db/2020.01.02
Successfully filled missing tables to partition: :/var/folders/l8/t7s11kcs02x3dchm9_m48mq80000gn/T/tmp2ts68edc/db/2020.01.01


Now that the database has resolved the missing tables within the partitions, we can view the new `quotes` table

In [27]:
db.quotes

,date,sym,open,high,low,close
,,,,,,
0,2020.01.03,AAPL,8.204026,0.9115201,3.916864,9.813545
1,2020.01.03,AAPL,8.092754,6.019578,0.08513137,2.825277
2,2020.01.03,AAPL,1.425043,8.881719,4.285461,7.820761
3,2020.01.03,AAPL,7.172736,3.33985,5.999403,3.010211
4,2020.01.03,AAPL,2.974185,1.559372,2.76356,5.182052
5,2020.01.03,AAPL,3.200759,7.485088,7.928813,6.437041
6,2020.01.03,AAPL,7.749599,5.559444,0.3300404,9.424896
7,2020.01.03,AAPL,4.885961,4.677432,8.288318,4.366883
8,2020.01.03,AAPL,7.412891,5.082189,9.214036,7.900838


Finally, to view the amount of saved data you can count the number of rows per partition using `partition_count`

In [28]:
db.partition_count()

,quotes,trades
,,
2020.01.01,0,500425
2020.01.02,0,499575
2020.01.03,10000,10000


## Cleanup temporary database created

In [29]:
tempdir.cleanup()

---